In [447]:
from random import choice
from tqdm.auto import tqdm
import numpy as np
from dataclasses import dataclass
import heapq

In [448]:
#initial settings
PUZZLE_DIM = 3
#action = namedtuple('Action', ['pos1', 'pos2'])
SOLUTION = np.array([n+1 for n in range(PUZZLE_DIM*PUZZLE_DIM)])
SOLUTION[-1] = 0
puzzle = SOLUTION[:]
print(SOLUTION)

[1 2 3 4 5 6 7 8 0]


In [ ]:

#utility functions and structures

#T = TypeVar('T')

@dataclass
class State:
    board: np.array
    gn: int = 0
    fn: int = np.inf
    #priority: int = np.inf
    __eq__ = lambda self, other: (np.array_equal(self.board, other.board)) #and self.gn == other.gn)
    __lt__ = lambda self, other: np.random.choice([True, False]) if self.fn == other.fn else self.fn < other.fn
    
@dataclass
class Action:
    x1: int
    y1: int
    x2: int
    y2: int

class PriorityQueue:
    #initializes the queue as a heap queue
    def __init__(self):
        self.elements: list[tuple[float, State]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, x: State, priority: float):
        heapq.heappush(self.elements, (priority, x))
        
    def get(self) -> State:
        return heapq.heappop(self.elements)[1]

def print_board(state: State):
    state = state.board
    matrix = np.resize(state, (PUZZLE_DIM, PUZZLE_DIM))
    print(matrix)

In [450]:
def available_actions(state: State) -> list['Action']:
    #find the empty tile
    board = state.board
    board = np.resize(board, (PUZZLE_DIM, PUZZLE_DIM))
    ii = np.where(board == 0)
    x= ii[0][0]
    y= ii[1][0]
    #print(x,y)
    #print_board (state)
    actions = list()
    if x > 0:
        actions.append(Action(x, y, x - 1, y))
    if x < PUZZLE_DIM - 1:
        actions.append(Action(x, y, x + 1, y))
    if y > 0:
        actions.append(Action(x, y, x, y - 1))
    if y < PUZZLE_DIM - 1:
        actions.append(Action(x, y, x, y + 1))
    return actions



def do_action(state: State, action: 'Action') -> State:
    new_board = state.board.copy()
    new_board[action.x1, action.y1], new_board[action.x2, action.y2] = new_board[action.x2, action.y2], new_board[action.x1, action.y1]
    new_state = State(new_board, state.gn + 1, 0)
    #print_board(new_state)
    return new_state

In [451]:
RANDOMIZE_STEPS = 100000
state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)),0,0)
#print_board(state)
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
print_board(state)

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

[[5 6 7]
 [8 4 3]
 [2 1 0]]


In [452]:
def compute_hcost_manhattan(state: State) -> int:
    board = state.board
    board = np.resize(board, (PUZZLE_DIM, PUZZLE_DIM))
    distance = 0
    solution = np.resize(SOLUTION, (PUZZLE_DIM, PUZZLE_DIM))
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if board[i, j] == 0:
                continue
            ii = np.where(solution == board[i, j])
            x= ii[0][0]
            y= ii[1][0]
            x, y = divmod(board[i, j] - 1, PUZZLE_DIM)
            distance += abs(x - i) + abs(y - j)
    return distance

compute_hcost_manhattan(state)

np.int64(18)

In [453]:
prova_heap = PriorityQueue()
best = State(SOLUTION, 0,0)
prova_heap.put(state, state.gn+compute_hcost_manhattan(state))
prova_heap.put(best, best.gn + compute_hcost_manhattan(best))

primo = prova_heap.get()
print_board(primo)
secondo = prova_heap.get()
print_board(secondo)
secondo.gn

stato_prova = State(np.array([1, 2, 3, 4, 5, 6, 7, 8, 0]), 0, 0)
np.array_equal(stato_prova.board, SOLUTION)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
[[5 6 7]
 [8 4 3]
 [2 1 0]]


True

In [454]:

def A_star_search(initial_state: State):
    
    solution = np.resize(SOLUTION, (PUZZLE_DIM, PUZZLE_DIM))
    initial_state.gn = 0
    initial_state.fn = compute_hcost_manhattan(initial_state)
    frontier = PriorityQueue()
    frontier.put(initial_state, initial_state.fn)
    visited = list()
    
    total_cost=1
    path_steps = 1
    
    i=0
    while not frontier.empty():
        
        #take the state with the lowest fscore from the frontier
        current = frontier.get()
        
        #goal test
        if np.array_equal(current.board, solution):
            return current, total_cost, path_steps

        for action in available_actions(current):
            #implicitly calculates gn for the new state in the do action
            new_state = do_action(current, action)
            new_state.fn = new_state.gn + compute_hcost_manhattan(new_state)
            total_cost+=1

            in_frontier = False
            for state in frontier.elements:
                if np.array_equal(state[1].board, new_state.board) and new_state.gn < state[1].gn:
                    in_frontier = True
                    state[1].gn = new_state.gn
                    break
            if not in_frontier and new_state not in visited:
                frontier.put(new_state, new_state.fn)
          
        visited.append(current)                 
        path_steps+=1
        
        # i+1
        # if i%500 == 0:
        #     print_board(current)
        #     print('Current manhattan distance:', current.fn - current.gn)
        
    return current, total_cost, path_steps


In [455]:
state = State(np.array([0,1,6,3,8,7,4,5,2]), 0, 0)
state.board = np.resize(state.board, (PUZZLE_DIM, PUZZLE_DIM))

print("=== INITIAL STATE ===")
print_board(state)

solution, cost, path_steps = A_star_search(state)
print("=== SOLUTION FOUND ===")
print_board(solution)
print("cost: ", cost)
print("path_steps: ", path_steps)

=== INITIAL STATE ===
[[0 1 6]
 [3 8 7]
 [4 5 2]]
=== SOLUTION FOUND ===
[[1 2 3]
 [4 5 6]
 [7 8 0]]
cost:  2581
path_steps:  967
